In [5]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
args={"batch_size":64,
"test_batch_size":1000,
"epochs":1,
"lr":0.03,
"gamma":0.7,
"seed":1,
"log_interval":100}

In [8]:
device = torch.device("cpu")

In [12]:
train_kwargs = {'batch_size': args["batch_size"]}
test_kwargs = {'batch_size': args["test_batch_size"]}

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


In [14]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args["lr"])

scheduler = StepLR(optimizer, step_size=1, gamma=args["gamma"])
for epoch in range(1, args["epochs"] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()




Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312112
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.610514
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.356229
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.568896
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.300628
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.357014
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.270445
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.472591
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.429579
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.450574

Test set: Average loss: 0.1949, Accuracy: 9416/10000 (94%)



In [18]:
import os

model_save_dir="pytorch_models"
model_save_path="{}/{}".format(model_save_dir, "mnist.pt")
model_script_save_path="{}/{}".format(model_save_dir, "model_scripted.pt")
os.makedirs(model_save_dir, exist_ok=True)
torch.save(model.state_dict(), model_save_path)

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(model_script_save_path) # Save

In [20]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import os


#this loading method needs model class code, so avoid.
# model = Net()
# model.load_state_dict(torch.load(model_save_path))
# model.eval()


#load model without class prototype
model = torch.jit.load(model_script_save_path)
model.eval()

test(model, device, test_loader)


Test set: Average loss: 0.1806, Accuracy: 9439/10000 (94%)

